In [1]:
import numpy as np
import torch
import librosa
import pandas as pd
import os
import random

In [ ]:
import numpy as np
import torch
import librosa
import pandas as pd
import os
import random
dataset_path = r'Z:\抑郁症\DAIC-WOZ Database'
folder_list = []
audio_files = []
audio_paths = []
transcript_paths = []

list_dir_dataset_path = os.listdir(dataset_path)
list_dir_dataset_path.sort()
for i in list_dir_dataset_path:
    if i.endswith('_P'):
        folder_list.append(i)
        for j in os.listdir(os.path.join(dataset_path, i)):
            if 'wav' in j:
                audio_files.append(j)
                audio_paths.append(os.path.join(dataset_path, i, j))
            if 'TRANSCRIPT' in j:
                if 'lock' in j or '._' in j:
                    pass
                else:
                    transcript_paths.append(os.path.join(dataset_path, i, j)

                    
def raw_features(audio_file, transcript_path):
    audio, sr = librosa.load(audio_file)
    ID = os.path.split(audio_file)[-1].split('_')[0]
    t_df = pd.read_csv(transcript_path, sep='\t')     #DEPA
    t_df = t_df[t_df.speaker=='Participant']
    start_end = t_df.loc[:, ['start_time', 'stop_time']].values
    win_length = int(100 * sr / 1000)
    hop_length = win_length // 4
    features = []
    dur_time = []

    for start, end in start_end: 
        dur_time.append(end-start)
        if end - start < 5:
            continue
        xxx = audio[int(start * sr):int(end * sr)]
#         print(len(xxx))
        feature1 = librosa.feature.mfcc(y=xxx,
                                   n_fft=win_length, 
                                   hop_length=hop_length).T + 1e-12
        feature2 = librosa.feature.zero_crossing_rate(y=xxx,
                                       frame_length=win_length, 
                                       hop_length=hop_length).T + 1e-12
        feature3 = librosa.feature.chroma_cqt(y=xxx,
                                             sr = sr,
                                             hop_length=hop_length).T + 1e-12
#         if feature1.shape[0]==feature2.shape[0]==feature3.shape[0]:
        feature = np.concatenate([feature1[:200, :],feature2[:200, :], 
                                  feature3[:200, :]],axis = 1)
        features.append(feature)
        
    return ID, features, dur_time

all_feature = []
k = []
flod_name = []
dur_time = []
for i in range(len(audio_files)):
    print("{}================".format(i))
    flod, features, times = raw_features(audio_paths[i], transcript_paths[i])
    k.append(len(features))
    all_feature.append(features)
    flod_name.append(flod)
    dur_time.append(times)
    print("num of {} is {}".format(i, len(features)))
    
path = r"C:\Users\sun\Desktop\models\features"
np.save(os.path.join(path, 'all_features'), all_feature)
np.save(os.path.join(path, 'flod_name'), flod_name)

In [8]:
all_feature = np.load(r'C:\Users\sun\Desktop\models\features\all_features_30s.npy', allow_pickle=True)
flod_name = np.load(r'C:\Users\sun\Desktop\models\features\flod_name.npy', allow_pickle=True)

In [ ]:
df = pd.read_csv(r'Z:\抑郁症\DAIC-WOZ Database\all_labels.csv') 

In [4]:
n = 0
for i in range(len(flod_name)):
    if flod_name[i] == str(df['ids'][i]):
        n += 1
n

189

In [5]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import RandomUnderSampler

data1 = []
data2 = []
data3 = []
label1 = []
label2 = []
label3 = []
k = 0

for i in range(len(flod_name)):
    if flod_name[i] == '385':
        continue
    
    x = all_feature[i]
    y = df['label'][i]
    
    if df['flod'][i] == 'train':
        for j in range(len(x)):
            xx = x[j][:200, :]
            data1.append(xx)
            label1.append(y)
            
    elif df['flod'][i] == 'val':
        for j in range(len(x)):
            xx = x[j][:200, :]
            data2.append(xx)
            label2.append(y)
        
    elif df['flod'][i] == 'test':
        for j in range(len(x)):
            xx = x[j][:200, :]
            data3.append(xx)
            label3.append(y)
        
data1 = np.array(data1)
data2 = np.array(data2)
data3 = np.array(data3)

In [ ]:
all_data = []
all_label = []

xx = data1.reshape(data1.shape[0], -1)
cc = RandomUnderSampler(random_state=42)
x, y = cc.fit_resample(xx, label1)
x = x.reshape(-1, data1.shape[1], data1.shape[2])
for i in range(len(x)):
    all_data.append(x[i])
    all_label.append(y[i])
    
xx = data2.reshape(data2.shape[0], -1)
cc = RandomUnderSampler(random_state=42)
x, y = cc.fit_resample(xx, label2)
x = x.reshape(-1, data2.shape[1], data2.shape[2])
for i in range(len(x)):
    all_data.append(x[i])
    all_label.append(y[i])

xx = data3.reshape(data3.shape[0], -1)
cc = RandomUnderSampler(random_state=42)
x, y = cc.fit_resample(xx, label3)
x = x.reshape(-1, data3.shape[1], data3.shape[2])
for i in range(len(x)):
    all_data.append(x[i])
    all_label.append(y[i])

In [13]:
path = r"C:\Users\sun\Desktop\models\features"
np.save(os.path.join(path, 'all_data_daic'), all_data)
np.save(os.path.join(path, 'all_label_daic'), all_label)